# Running Auto Correlation with Talon Deployer and BITE
###### Last Updated: 09/04/24

This demo will show the basic operation of auto correlation using the new Deployer and BITE TANGO devices currently deployed in this repository. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

## Prerequisites


First, this notebook assumes you have a running environment launched from a pipeline, in particular it assumes you are running off one launched from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline. Secondly, for ease of dev work, it also assumes you are using a virtual env. This notebook was made with Python 3.10 in mind.

Finally, make sure all requirements are installed via [poetry](https://python-poetry.org/docs/basic-usage/#installing-dependencies), and after that we can grab the imports required.

In [1]:
import os
from PyTango import DeviceProxy, Database
from time import sleep
import json

### Running on PSI


To Run on PSI, once running the launch step from the PSI pipeline, grab your booted namespace's name. You can run a check on the ns as well to make sure the pods are ready.

In [2]:
!kubectl get ns | grep ska-mid-psi

ci-ska-mid-psi-1245030042-jbmorgado                  Active   8h
ci-ska-mid-psi-1245030042-jbmorgado-sdp              Active   8h
ci-ska-mid-psi-1245030042-maecst                     Active   17h
ci-ska-mid-psi-1245030042-maecst-sdp                 Active   17h
ci-ska-mid-psi-1246254401-alexschell                 Active   8m53s
ci-ska-mid-psi-1246254401-alexschell-sdp             Active   9m33s


To make future work a bit easier we can load this into a NS var.

In [3]:
# Change this var to be the non-spd namespace with your username in it
NS = "ci-ska-mid-psi-1246254401-alexschell"

url = "https://142.73.34.170/"+NS+"/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run"
print("You can monitor board status using: https://142.73.34.170/{}/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run".format(NS))

You can monitor board status using: https://142.73.34.170/ci-ska-mid-psi-1246254401-alexschell/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run


At this point, all components should be in the `disabled` state.

We can then check to ensure the pods have spun up successfully.

In [ ]:
!kubectl -n $NS get pods

## Initial Setup


First we want to set up the target board that we will be using for the auto correlation. While this can be multiple boards, for now we will only need one. Punch out the board(s) you have access to and set the `TARGET_BOARD` to assign it to be used in future steps.

The `TANGO_HOST` will be created based of the namespace you set earlier.

In [4]:
# Set to the boards you wish to configure.
TARGET_BOARD = [7]

TANGO_HOST = "databaseds-tango-base."+NS+".svc.cluster.local:10000"
print("Will be using HOST: ", TANGO_HOST)
os.environ["TANGO_HOST"] = TANGO_HOST

Will be using HOST:  databaseds-tango-base.ci-ska-mid-psi-1246254401-alexschell.svc.cluster.local:10000


We then load in the locations of local files for configuration. If you wish to use custom files you can change the folders pointed to, otherwise this code block can be run without edits.

In [5]:
# Get files from configs folder in repo
CURRDIR = os.path.dirname(os.getcwd())
CORRELATOR_FILE_LOCATION = os.path.abspath(os.path.join(CURRDIR,"configs/auto-correlation"))
COMMON_FILE_LOCATION = os.path.abspath(os.path.join(CURRDIR,"configs/common"))

Next, we pass in the SLIM mesh config files by copying them to the namespace, these files should be in the json_files storage folder. Custom files here are not required, but if needed, the following two code blocks can be used.

In [6]:
# Change this value if you want to specify a certain slim directory for files
slim_fs_config = ""
slim_vis_config = ""
if slim_fs_config != "" and slim_vis_config != "" :
    fs_config_path = os.path.join(CORRELATOR_FILE_LOCATION, slim_fs_config)
    vis_config_path = os.path.join(CORRELATOR_FILE_LOCATION, slim_vis_config)    
else:
    print("SLIM will use defaults for this test.")

SLIM will use defaults for this test.


In [ ]:
!"kubectl cp {fs_config_path} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml"
!"kubectl cp {vis_config_path} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml"

For this demo, we will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them as we would in the UI.

In [7]:
# Setup the device proxies targeting bite and deployer
db = Database()

deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")
#Check the devices initially deployed to the database
print(*db.get_device_exported("*").value_string, sep="\n")
# Make sure the deployer device is set to ON
deployer_tango.On()
print(deployer_tango.state())

dserver/CbfController/controller
dserver/CbfSubarray/cbfsubarray-01
dserver/CbfSubarray/cbfsubarray-02
dserver/CbfSubarray/cbfsubarray-03
dserver/DataBaseds/2
dserver/ECBite/bite
dserver/ECDeployer/deployer
dserver/FspMulti/fsp-01
dserver/FspMulti/fsp-02
dserver/FspMulti/fsp-03
dserver/FspMulti/fsp-04
dserver/MidCspCapabilityFsp/capabilityfsp
dserver/MidCspCapabilityVcc/capabilityvcc
dserver/MidCspController/controller
dserver/MidCspSubarray/subarray1
dserver/MidCspSubarray/subarray2
dserver/MidCspSubarray/subarray3
dserver/PowerSwitch/powerswitch-001
dserver/PowerSwitch/powerswitch-002
dserver/PowerSwitch/powerswitch-003
dserver/PowerSwitch/powerswitch-004
dserver/SDPController/0
dserver/SDPQueueConnector/01
dserver/SDPSubarray/01
dserver/Slim/mesh
dserver/SlimLink/fs-links
dserver/SlimLink/vis-links
dserver/TalonBoard/talon-001
dserver/TalonBoard/talon-002
dserver/TalonBoard/talon-003
dserver/TalonBoard/talon-004
dserver/TalonBoard/talon-005
dserver/TalonBoard/talon-006
dserver/Talon

Next we load in the hardware configuration depending on the talon boards selected. If a higher number board is chosen, we need to use the swapped config file.

In [30]:
if any(i > 4 for i in TARGET_BOARD):
    print("Using swap for higher number talons")
    config = "hw_config/hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    TARGET_BOARD = list(map(lambda x:x-4,TARGET_BOARD))
else:
    print("Using standard HW config")
    config = "hw_config/hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(COMMON_FILE_LOCATION, config)

Using standard HW config


Now we copy this file into the controller pod:

In [31]:
!kubectl cp $HW_CONFIG_FILE $NS/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

## Deploying Using the Command Line (Old Method)

This is the old method of downloading artifacts and deploying them to the DS database, use it in case the new Deployer device doesn't work.

In [35]:
deployer_board_target = TARGET_BOARD[0]
print("Talon board -> {}".format(deployer_board_target))
!kubectl exec -ti -n $NS ec-deployer -- python3 midcbf_deployer.py --generate-talondx-config --boards=$deployer_board_target
!kubectl exec -ti -n $NS ec-deployer -- python3 midcbf_deployer.py --download-artifacts
!kubectl exec -ti -n $NS ec-deployer -- python3 midcbf_deployer.py --config-db

Talon board -> 3


[talondx.py: line 487]INFO: User: root
[talondx.py: line 542]INFO: Generate talondx-config.json file
[talondx.py: line 487]INFO: User: root
[talondx.py: line 536]INFO: Download Artifacts
[talondx.py: line 330]INFO: Conan version: 1.62.0
[talondx.py: line 332]INFO: Conan local cache: There are no packages
[talondx.py: line 333]INFO: Clearing Conan local cache... WARN: No package recipe matches '*'
[talondx.py: line 336]INFO: Conan local cache: There are no packages
[talondx.py: line 339]INFO: DS Binary: ds-hps-master
[talondx.py: line 344]INFO: Conan info: {'package_name': 'dshpsmaster', 'user': 'nrc', 'channel': 'stable', 'version': '1.0.6', 'profile': 'conan_aarch64_profile.txt'}
Conan 1 is on a deprecation path, please consider migrating to Conan 2
Configuration:
[settings]
arch=armv8
build_type=Release
compiler=gcc
compiler.libcxx=libstdc++11
compiler.version=7
os=Linux
[options]
[build_requires]
[env]
AR=aarch64-linux-gnu-ar
AS=aarch64-linux-gnu-as
CC=aarch64-linux-gnu-gcc
CHOST=aa

Ensure that the download steps runs, and reaches completion.

## Deploying Using the Deployer Device

First, we set the target talon boards we want to set up our configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted.

In [23]:
deployer_tango.targetTalons = TARGET_BOARD
print(deployer_tango.targetTalons)

[3]


With this set, we can then run the configuration command by calling the generate_config_jsons command.

In [24]:
deployer_tango.generate_config_jsons()

We then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

In [25]:
deployer_tango.set_timeout_millis(200000)
try:
    deployer_tango.download_artifacts()
except Exception as e:
    print(e)
    print("Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes.")
deployer_tango.set_timeout_millis(4500)

To check that the artifacts downloaded successfully, we want to check that the following returns something like `INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading`. 

In [28]:
!kubectl logs -n $NS ds-deployer-deployer-0 | grep 'Finished downloading'

Finally, we can configure the TANGO database with all the tango devices we just downloaded using the ConfigDB command.

In [ ]:
deployer_tango.configure_db()

The TANGO database should now be configured with all the devices needed for the next step, running the BITE device.

In [ ]:
print(*db.get_device_exported("*").value_string, sep="\n")

## Uploading Controller Settings and Starting the Boards

Now that we have the requried devices deployed and exported, we can set up the DeviceProxies for the devices we'll use in the next steps:

In [36]:
control_tango = DeviceProxy("mid-csp/control/0")
print(control_tango.status())
subarray_tango = DeviceProxy("mid-csp/subarray/01")
print(subarray_tango.status())
CBF_tango = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(CBF_tango.status())

The device is in DISABLE state.
The device is in DISABLE state.
The device is in DISABLE state.


**Before running this step and the following steps, ensure that you have checked out the boards you have selected to use!**
***

We can then set adminMode to 0 (ONLINE), allowing us to run commands, and set simulationMode to 0 (FALSE).

In [37]:
# Set relevant values on the mid-csp controller
control_tango.adminMode = 0 
control_tango.write_attribute("cbfSimulationMode", 0)
control_tango.cbfSimulationMode = 1

if control_tango.read_attribute("adminMode").value == 0 and control_tango.read_attribute("cbfSimulationMode").value == 0:
    print ("Set values successfully!")
elif control_tango.read_attribute("adminMode").value == 0 and control_tango.read_attribute("cbfSimulationMode").value == 1:
    print ("Set to simulation mode on.")
else:
    print ("Error, couldn't set values!")

Set to simulation mode on.


Checking the status dashboard, it should now display that all devices are both OFF and that the simulationstate is FALSE.

 Next, we load in a inital values parameters and pass it to the controller, to do this we read in JSON file and pass it as a DevString to the relevant device command: 

In [38]:
init_sys_param_file = os.path.join(COMMON_FILE_LOCATION, "sys_params/initial_system_param.json")

with open(init_sys_param_file) as init_file:
    data = json.load(init_file)
upload_result = CBF_tango.InitSysParam(json.dumps(data))
print(upload_result[1])

['CbfController InitSysParam command completed OK']


Now, we turn ON the Controller by passing it the device we want to turn on, and letting it run for 45ms to give the boards time to power on.

In [39]:
print(control_tango.status())
control_tango.set_timeout_millis(50000)
target = ["mid_csp_cbf/sub_elt/controller"]
control_tango.On(target)
sleep(55)
print(control_tango.status())

The device is in OFF state.
The device is in ON state.


After running this step, check with the Tarnata dashboard to check that the boards are started.

## Running Commands in BITE Through Comamnd Line Args (Old Method)

This is the older method for running BITE commands, utilizing kubectl to run python files directly:

First we configure, specificing the MAC adress of the boards and the board # to configure.

In [ ]:
target_mac_address = "08:c0:eb:9d:47:78"
standin_board = TARGET_BOARD[0]
!kubectl exec -ti -n $NS ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards=$standin_board --bite_mac_address=$target_mac_address

Then we start LSTV replay, check the logs and get the epoch value to use later (`INFO: start_utc_time_offset = start_utc_time.unix_tai - ska_epoch_tai = <EPOCH VALUE TO COPY>`)

In [ ]:
!kubectl exec -ti -n $NS ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards=$standin_board

## Running Commands Through the BITE Device

Now that the BITE tango Device has been deployed via the deployer, we can use it to configure tests. First we check the device is running.

In [ ]:
target_mac_address = "08:c0:eb:9d:47:78"
bite_tango = DeviceProxy("mid_csp_cbf/ec/bite")
# Running this should return RUNNING
print(bite_tango.State())

Now that we have confirmed it is running, we load in what board we want to configure.

In [ ]:
bite_tango.boards = TARGET_BOARD
bite_tango.bite_mac_address = target_mac_address

For now, we can use the defaults and simply call the write command for the test configs. This should return the configuration for each board passed in.

In [ ]:
#print(bite_tango.command_inout("generate_bite_data"))

# For now run the backup
standin_board = TARGET_BOARD[0]
print(standin_board)
!kubectl exec -ti -n $NS ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards=$standin_board --bite_mac_address=$target_mac_address

In [ ]:
!kubectl -n $NS logs ds-bite-bite-0 | grep generate_bite_data

With the bite data generated, we can then start generating LSTV playback data:

In [ ]:
bite_tango.start_lstv_replay()

We can then check the logs to ensure the LSTV geneartion runs correctly.

In [ ]:
!kubectl -n $NS logs ds-bite-bite-0 | grep start_lstv_replay

## Assigning Resources

Next, we use the relevant subarray device to assign resources. First, as with the other devices, we establish a DeviceProxy to connect to it. We also read in the assign_resources file to load in using the command. 

For the loaded data, we must ensure the right board is selected, modifying the data for the receptor ID based on the Talon board selected.

In [40]:
# Load in the data from the file.
subarray_tango = DeviceProxy("mid-csp/subarray/01")
assign_resources_file = os.path.join(CORRELATOR_FILE_LOCATION, "assign_resources.json")
with open(assign_resources_file) as init_file:
    config_dict = json.load(init_file)

# In order to use the correct receptor, we modify the assign_resources data to use the correct receptor based on the board we're using
receptor_map = ["SKA001","SKA036","SKA063","SKA100"]
print(config_dict)
config_dict['dish']['receptor_ids'] = list(map(lambda x:receptor_map[x-1],TARGET_BOARD))
print(json.dumps(config_dict))


{'interface': 'https://schema.skao.int/ska-csp-assignresources/2.2', 'subarray_id': 1, 'dish': {'receptor_ids': ['SKA001']}}
{"interface": "https://schema.skao.int/ska-csp-assignresources/2.2", "subarray_id": 1, "dish": {"receptor_ids": ["SKA063"]}}


Next,we run the actual command to pass in the resources:

In [41]:
subarray_tango.AssignResources(json.dumps(config_dict))
while subarray_tango.obsState != 2:
    pass
print(subarray_tango.obsState)

obsState.IDLE


## SDP Setup

Next, we run a few commands to set up the Science Data Processor (SDP), utilizing the sdp namespace

First, we get the files needed to set up the SDP, if you require custom files, load in new ones or change the commands to point to them.

In [45]:
# Change this if you want to use a different folder for files
SPD_FOLDER = CORRELATOR_FILE_LOCATION

sdp_assign_resources_file = os.path.join(SPD_FOLDER,"assign_resources_sdp.json")
sdp_configure_file = os.path.join(SPD_FOLDER,"configure_sdp.json")
sdp_scan_file = os.path.join(SPD_FOLDER,"scan_sdp.json")
if not os.path.isfile(sdp_scan_file) or not os.path.isfile(sdp_configure_file) or not os.path.isfile(sdp_assign_resources_file):
    print("Warning, one or more of the files pointed to doesn't exist!")
else: 
    print("All files loaded correctly")

All files loaded correctly


Next, we set up a proxy to the SDP device:

In [50]:
sdp_tango = DeviceProxy("test-sdp/subarray/01")
sdp_tango.On()
print(sdp_tango.Status())

The device is in ON state.


Now, we assign resources as we did with the subarray, as with the csp, we map the receptor boards based on our checked out board.

In [51]:
with open(sdp_assign_resources_file) as f:
    sdp_resources = json.load(f)
print(sdp_resources)
sdp_resources['resources']['receptors'] = list(map(lambda x:receptor_map[x-1],TARGET_BOARD))
print(json.dumps(sdp_resources))

{'interface': 'https://schema.skao.int/ska-sdp-assignres/0.4', 'resources': {'csp_links': [1, 2, 3, 4], 'receptors': ['SKA001'], 'receive_nodes': 1}, 'execution_block': {'eb_id': 'eb-test-20210630-00019', 'context': {}, 'max_length': 21600.0, 'channels': [{'channels_id': 'vis_channels', 'spectral_windows': [{'spectral_window_id': 'fsp_1_channels', 'count': 100, 'start': 0, 'stride': 1, 'freq_min': 296368100.0, 'freq_max': 297712100.0, 'link_map': [[0, 0], [200, 1], [744, 2], [944, 3]]}]}], 'polarisations': [{'polarisations_id': 'all', 'corr_type': ['XX', 'XY', 'YY', 'YX']}], 'fields': [{'field_id': 'field_a', 'phase_dir': {'ra': [2.711325], 'dec': [-0.01328889], 'reference_time': '...', 'reference_frame': 'ICRF3'}, 'pointing_fqdn': 'low-tmc/telstate/0/pointing'}, {'field_id': 'field_b', 'phase_dir': {'ra': [12.48519], 'dec': [2.052388], 'reference_time': '...', 'reference_frame': 'ICRF3'}, 'pointing_fqdn': 'low-tmc/telstate/0/pointing'}], 'beams': [{'beam_id': 'vis0', 'function': 'visi

And run the command to assign resources:

In [52]:
sdp_tango.AssignResources(json.dumps(sdp_resources))

Next we send the configure command along with the required config file: (This step may fail if run right after the previous one, wait before running )

In [ ]:
print(sdp_tango.obsState)

In [54]:
with open(sdp_configure_file) as f:
    sdp_configuration = json.load(f)
sdp_tango.Configure(json.dumps(sdp_configuration))

DevFailed: DevFailed[
DevError[
    desc = Command not allowed in obsState RESOURCING
  origin = command_allowed_obs_state
  reason = API_CommandNotAllowed
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device test-sdp/subarray/01, command Configure
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at (/src/cppTango/cppapi/client/devapi_base.cpp:1338)
  reason = API_CommandFailed
severity = ERR]
]

We then start the SDP output via the Scan command:

In [ ]:
with open(sdp_scan_file) as f:
    sdp_scan = json.load(f)
sdp_tango.Scan(json.dumps(sdp_scan))

After running this step, a pod for reciving visabilities will be launched, grab the IP of this pod and place it in the output_host var in the configure_scan json file.

In [44]:
# Grab the pod name
!kubectl -n $NS-sdp get pods | grep vis-receive

No resources found in ci-ska-mid-psi-1246254401-alexschell-sdp namespace.


In [42]:
# Then find the IP:
vis_pod = "proc-pb-test-20211111-00019-vis-receive-00-0"
!kubectl -n $NS-sdp describe pod $vis_pod | grep net1 -A 3

Error from server (NotFound): pods "proc-pb-test-20211111-00019-vis-receive-00-0" not found


With this pod, we will also want to prep for mointoring the pod for when it (hopefully) recivies the visibilties

Run this command, then use the output in a seperate terminal to enter the correct pod:

In [43]:
!echo kubectl exec -n $NS-sdp -ti $vis_pod -- bash

kubectl exec -n ci-ska-mid-psi-1246254401-alexschell-sdp -ti proc-pb-test-20211111-00019-vis-receive-00-0 -- bash


We can then run the following commands in this pod to start monitoring for the correct traffic:

Now on the CBF side we can load in the corresponding Scan file.

In [46]:
cbf_configure_scan_file = os.path.join(CORRELATOR_FILE_LOCATION,"configure_scan.json")
cbf_scan_file  = os.path.join(CORRELATOR_FILE_LOCATION,"scan.json")
with open(cbf_configure_scan_file) as f:
    cbf_config = json.load(f)
with open(cbf_scan_file) as f:
    cbf_scan = json.load(f)

In [47]:
print(json.dumps(cbf_config))

{"interface": "https://schema.skao.int/ska-csp-configure/2.4", "subarray": {"subarray_name": "4 receptors"}, "common": {"config_id": "4 receptor, band 1, 1 FSP, no options", "frequency_band": "1", "subarray_id": 1}, "cbf": {"delay_model_subscription_point": "ska_mid/tm_leaf_node/csp_subarray_01/delayModel", "fsp": [{"fsp_id": 1, "function_mode": "CORR", "frequency_slice_id": 1, "zoom_factor": 0, "integration_factor": 10, "channel_offset": 0, "output_link_map": [[0, 1]], "output_host": [[0, "10.50.1.34"]], "output_port": [[0, 21000, 1]]}]}}


In [48]:
print("Observation state: {}".format(subarray_tango.obsState))
subarray_tango.Configure(json.dumps(cbf_config))
sleep(5)
print("Observation state: {}".format(subarray_tango.obsState))

Observation state: 2
Observation state: 4


As a temporary step, load this into the delay model on the CSP Subarray dash make sure to change the `epoch` value to the one generated by the BITE LSTV replay start command, and the `receptor` to match the one set :

In [ ]:
delay_model ={"interface":"https://schema.skao.int/ska-csp-delaymodel/2.2","epoch":765929670.5153561,"validity_period":400.0,"delay_details":[{"receptor":"SKA063","poly_info":[{"polarization":"X","coeffs":[0.0,0.0,0.0,0.0,0.0,0.0]},{"polarization":"Y","coeffs":[0.0,0.0,0.0,0.0,0.0,0.0]}]}]}
delayModelProxy = DeviceProxy('ska_mid/tm_leaf_node/csp_subarray_01')
delayModelProxy.write_attribute('delayModel', json.dumps(delay_model))

And then the matching command in the subarray:

In [ ]:
subarray_tango.Scan(json.dumps(cbf_scan))

In [ ]:
print("Observation state: {}".format(subarray_tango.obsState))

Checking the dashboard, the dish should now be in the SCANNING status.

## Checking Visabilties

With the devices scanning and linked, we can monitor the output via monitoring the network packets that come from it. Use the tcpdum running terminal to check that packet lengths are correct:

## Cleanup

Once we're satisfied with the reuslts, we can stop the scans and shut down the boards.

In [ ]:
sdp_tango.EndScan()
subarray_tango.EndScan()

In [ ]:
control_tango.Off(target)

If the boards are on, use the LRU command to turn them off.

Now that we're done, free up dev resources on PSI by deleting your ns.

In [ ]:
!kubectl delete ns $NS
!kubectl delete ns $NS-sdp

In [ ]:
!kubectl get ns | grep ska-mid-psi